<a href="https://colab.research.google.com/github/brendan123/Semantic-Segmentation/blob/milestone-1/SemanticSegementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone 1: Environment Preparation

## Install Dependencies

In [1]:
# Seed repo Python dependencies
! pip install patchify
! pip install segmentation-models
! pip install -q keras # Keras

# Ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip # download ngrok and unzip it
! unzip ngrok-stable-linux-amd64.zip && rm ngrok-stable-linux-amd64.zip

# NNI
! pip install nni # install nni
! mkdir -p nni_repo
! git clone https://github.com/microsoft/nni.git nni_repo/nni # clone NNI's offical repo to get examples

#kaggle
!pip install -q kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.7 MB/s 
--2022-10-27 21:23:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  13.8MB/s    in 1.0s    

2022-10-27 21:23:50 (13.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/s

## Cloning Repo into Workspace

In [2]:
# Clone seed repo
! git clone https://github.com/bnsreenu/python_for_microscopists
! mkdir "seed_repo"
! mv ./python_for_microscopists/228_semantic_segmentation_of_aerial_imagery_using_unet/* ./seed_repo/
! rm -rf python_for_microscopists

Cloning into 'python_for_microscopists'...
remote: Enumerating objects: 1352, done.
remote: Counting objects: 100% (513/513), done.
remote: Compressing objects: 100% (342/342), done.
remote: Total 1352 (delta 170), reused 445 (delta 158), pack-reused 839
Receiving objects: 100% (1352/1352), 109.39 MiB | 20.53 MiB/s, done.
Resolving deltas: 100% (441/441), done.
Checking out files: 100% (640/640), done.


## Downloading Dataset from Kaggle

In [3]:
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 700 ~/.kaggle/kaggle.json
! kaggle datasets download -d humansintheloop/semantic-segmentation-of-aerial-imagery
! unzip semantic-segmentation-of-aerial-imagery.zip
! rm semantic-segmentation-of-aerial-imagery.zip
! rm kaggle.json # Delete kaggle.json

Saving kaggle.json to kaggle.json
 30% 9.00M/29.6M [00:00<00:00, 27.4MB/s]
100% 29.6M/29.6M [00:00<00:00, 71.3MB/s]
Archive:  semantic-segmentation-of-aerial-imagery.zip
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_001.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_002.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_003.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_004.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_005.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_006.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_007.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_008.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_009.jpg  
  inflating: Semantic segmentation dataset/Tile 1/masks/image_part_001.png  
  inflating: Semantic segmentation dataset/Tile 1/m

## NNI UI

In [4]:
# Kills old ngrok instance running on 5000 (if running)
! kill $(lsof -t -i:5000)
# This is my authtoken please don't take it
files.upload() # upload ngrok.txt containing authtoken
! ./ngrok authtoken "$(<ngrok.txt)"
! rm ngrok.txt

# Start an NNI example
! nnictl create --config nni_repo/nni/examples/trials/mnist-pytorch/config.yml --port 5000 &
# Start ngrok
get_ipython().system_raw('./ngrok http 5000 &')

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


Saving ngrok.txt to ngrok.txt
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
[2022-10-27 21:24:53] Creating experiment, Experiment ID: 9d07vwmn
[2022-10-27 21:24:53] Starting web server...
[2022-10-27 21:24:54] Setting up...
[2022-10-27 21:24:54] Web portal URLs: http://127.0.0.1:5000 http://172.28.0.2:5000
[2022-10-27 21:24:54] To stop experiment run "nnictl stop 9d07vwmn" or "nnictl stop --all"
[2022-10-27 21:24:54] Reference: https://nni.readthedocs.io/en/stable/reference/nnictl.html
{"tunnels":[],"uri":"/api/tunnels"}


In [10]:
# Get ngrok tunnel URL
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://ac6f-34-90-38-30.ngrok.io


## Python Imports Verification

In [5]:
# Verify Python Dependencies are installed
import os
import cv2
import numpy as np
import pandas as pd
from patchify import patchify
from matplotlib import pyplot
from PIL import Image
import segmentation_models as sm
from keras.metrics import MeanIoU

# No import errors

Segmentation Models: using `keras` framework.
